# Helmholtz equation on a spherical shell

## Using spherical coordinates

Helmholtz equation is given as

$$
-\nabla^2 u + \alpha u = f
$$

In [1]:
import os
from shenfun import *
from shenfun.la import SolverGeneric1NP
import sympy as sp

Define spherical coordinates $(r, \theta, \phi)$

$$
\begin{align}
x &= r \sin \theta \cos \phi \\
y &= r \sin \theta \sin \phi \\
z &= r \cos \theta
\end{align}
$$

In [2]:
r = 1
theta, phi = sp.symbols('x,y', real=True, positive=True)
psi = (theta, phi)
rv = (r*sp.sin(theta)*sp.cos(phi), r*sp.sin(theta)*sp.sin(phi), r*sp.cos(theta))

Define tensorproductspace, test- and trialfunction

In [3]:
N, M = 60, 40
alpha = 2
L0 = Basis(N, 'C', domain=(0, np.pi))
F1 = Basis(M, 'F', dtype='D')
T = TensorProductSpace(comm, (L0, F1), coordinates=(psi, rv))
v = TestFunction(T)
u = TrialFunction(T)

Use one spherical harmonic function as a manufactured solution

In [4]:
sph = sp.functions.special.spherical_harmonics.Ynm
ue = sph(6, 3, theta, phi)

Compute the right hand side on the quadrature mesh and take the scalar product

In [5]:
g = (-div(grad(u))+alpha*u).tosympy(basis=ue, psi=psi)
gj = Array(T, buffer=g)
g_hat = Function(T)
g_hat = inner(v, gj, output_array=g_hat)

Assemble matrices

In [6]:
mats = inner(v, -div(grad(u))+alpha*u)

Solve linear equation system

In [7]:
u_hat = Function(T)
Sol1 = SolverGeneric1NP(mats)
u_hat = Sol1(g_hat, u_hat)

Transform back to real space and compute the error.

In [8]:
uj = u_hat.backward()
uq = Array(T, buffer=ue)
print('Error =', np.linalg.norm(uj-uq))

Error = 1.7409506078302179e-09


Postprocessing. Refine for a nicer plot. Note that refine simply pads Functions with zeros, which gives more quadrature points. u_hat has NxM quadrature points, refine using any higher number.

In [9]:
u_hat2 = u_hat.refine([N*3, M*3])
ur = u_hat2.backward(uniform=True)

Plot solution using mayavi

In [10]:
from mayavi import mlab
mlab.init_notebook('x3d', 600, 600)

Notebook initialized with x3d backend.


In [11]:
xx, yy, zz = u_hat2.function_space().local_curvilinear_mesh(uniform=True)
# Wrap periodic direction around
if T.bases[1].domain == (0, 2*np.pi):
    xx = np.hstack([xx, xx[:, 0][:, None]])
    yy = np.hstack([yy, yy[:, 0][:, None]])
    zz = np.hstack([zz, zz[:, 0][:, None]])
    ur = np.hstack([ur, ur[:, 0][:, None]])
mlab.figure(bgcolor=(1, 1, 1), size=(400, 400))
m = mlab.mesh(xx, yy, zz, scalars=ur.real, colormap='jet')
m